# 1.2 SageMaker Training with Experiments

## 학습 작업의 실행 노트북 개요

- SageMaker Training에 SageMaker 실험을 추가하여 여러 실험의 결과를 비교할 수 있습니다.
    - [작업 실행 시 필요 라이브러리 import](#작업-실행-시-필요-라이브러리-import)
    - [SageMaker 세션과 Role, 사용 버킷 정의](#SageMaker-세션과-Role,-사용-버킷-정의)
    - [하이퍼파라미터 정의](#하이퍼파라미터-정의)
    - [학습 실행 작업 정의](#학습-실행-작업-정의)
        - 학습 코드 명
        - 학습 코드 폴더 명
        - 학습 코드가 사용한 Framework 종류, 버전 등
        - 학습 인스턴스 타입과 개수
        - SageMaker 세션
        - 학습 작업 하이퍼파라미터 정의
        - 학습 작업 산출물 관련 S3 버킷 설정 등
    - [학습 데이터셋 지정](#학습-데이터셋-지정)
        - 학습에 사용하는 데이터셋의 S3 URI 지정
    - [SageMaker 실험 설정](#SageMaker-실험-설정)
    - [학습 실행](#학습-실행)
    - [데이터 세트 설명](#데이터-세트-설명)
    - [실험 결과 보기](#실험-결과-보기)

### 작업 실행 시 필요 라이브러리 import

In [1]:
import boto3
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


### SageMaker 세션과 Role, 사용 버킷 정의

In [2]:
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

In [3]:
bucket = sagemaker_session.default_bucket()
code_location = f's3://{bucket}/xgboost/code'
output_path = f's3://{bucket}/xgboost/output'

### 하이퍼파라미터 정의

In [4]:
hyperparameters = {
       "scale_pos_weight" : "29",    
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}

### 학습 실행 작업 정의

In [5]:
instance_count = 1
instance_type = "ml.m5.large"
# instance_type = "local"
max_run = 1*60*60

use_spot_instances = False
if use_spot_instances:
    max_wait = 1*60*60
else:
    max_wait = None

In [6]:
if instance_type in ['local', 'local_gpu']:
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
else:
    sagemaker_session = sagemaker.session.Session()

In [16]:
from sagemaker.xgboost.estimator import XGBoost

estimator = XGBoost(
    entry_point="xgboost_script.py",
    source_dir="src",
    output_path=output_path,
    code_location=code_location,
    hyperparameters=hyperparameters,
    role=role,
    sagemaker_session=sagemaker_session,
    instance_count=instance_count,
    instance_type=instance_type,
    framework_version="1.7-1",
    max_run=max_run,
    use_spot_instances=use_spot_instances,  # spot instance 활용
    max_wait=max_wait,
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.large.


### 학습 데이터셋 지정

In [17]:
data_path=f's3://{bucket}/xgboost/dataset'
!aws s3 sync ../data/dataset/ $data_path

In [18]:
if instance_type in ['local', 'local_gpu']:
    from pathlib import Path
    file_path = f'file://{Path.cwd()}'
    inputs = file_path.split('lab_1_training')[0] + 'data/dataset/'
    
else:
    inputs = data_path
inputs

's3://sagemaker-us-west-2-322537213286/xgboost/dataset'

### SageMaker 실험 설정

In [19]:
!pip install -U sagemaker-experiments

In [20]:
experiment_name='xgboost-poc-1'

In [21]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from time import strftime

In [22]:
def create_experiment(experiment_name):
    try:
        sm_experiment = Experiment.load(experiment_name)
    except:
        sm_experiment = Experiment.create(experiment_name=experiment_name)

In [23]:
def create_trial(experiment_name):
    create_date = strftime("%m%d-%H%M%s")       
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{create_date}',
                            experiment_name=experiment_name)

    job_name = f'{sm_trial.trial_name}'
    return job_name

### 학습 실행

In [24]:
create_experiment(experiment_name)
job_name = create_trial(experiment_name)

estimator.fit(inputs = {'inputdata': inputs},
                  job_name = job_name,
                  experiment_config={
                      'TrialName': job_name,
                      'TrialComponentDisplayName': job_name,
                  },
                  wait=False)

INFO:sagemaker:Creating training-job with name: xgboost-poc-1-0912-23261726183606


In [25]:
estimator.logs()

2024-09-12 23:26:49 Starting - Starting the training job...
2024-09-12 23:27:02 Starting - Preparing the instances for training...
2024-09-12 23:27:33 Downloading - Downloading input data......
2024-09-12 23:28:23 Downloading - Downloading the training image.....[2024-09-12 23:29:29.680 ip-10-2-99-124.us-west-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-09-12 23:29:29.703 ip-10-2-99-124.us-west-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-09-12:23:29:30:INFO] Imported framework sagemaker_xgboost_container.training
[2024-09-12:23:29:30:INFO] No GPUs detected (normal if no gpus installed)
[2024-09-12:23:29:30:INFO] Invoking user training script.
[2024-09-12:23:29:30:INFO] Module xgboost_script does not provide a setup.py. 
Generating setup.py
[2024-09-12:23:29:30:INFO] Generating setup.cfg
[2024-09-12:23:29:30:INFO] Generating MANIFEST.in
[2024-09-12:23:29:30:INFO] Installing module with the following comma

UnexpectedStatusException: Error for Training job xgboost-poc-1-0912-23261726183606: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/miniconda3/bin/python3 -m xgboost_script --eta 0.2 --max_depth 3 --num_round 100 --objective binary:logistic --scale_pos_weight 29", exit code: 1. Check troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html

###  실험 결과 보기
위의 실험한 결과를 확인 합니다.
- 각각의 훈련잡의 시도에 대한 훈련 사용 데이터, 모델 입력 하이퍼 파라미터, 모델 평가 지표, 모델 아티펙트 결과 위치 등의 확인이 가능합니다.
- **아래의 모든 내용은 SageMaker Studio 를 통해서 직관적으로 확인이 가능합니다.**

In [26]:
from sagemaker.analytics import ExperimentAnalytics
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 10
pd.options.display.max_colwidth = 100

In [27]:
trial_component_training_analytics = ExperimentAnalytics(
    sagemaker_session= sagemaker_session,
    experiment_name= experiment_name,
    sort_by="metrics.validation:auc.max",        
    sort_order="Descending",
    metric_names=["validation:auc"]
)

trial_component_training_analytics.dataframe()[['Experiments', 'Trials', 'validation:auc - Min', 'validation:auc - Max',
                                                'validation:auc - Avg', 'validation:auc - StdDev', 'validation:auc - Last', 
                                                'eta', 'max_depth', 'num_round', 'scale_pos_weight']]

,Experiments,Trials,validation:auc - Min,validation:auc - Max,validation:auc - Avg,validation:auc - StdDev,validation:auc - Last,eta,max_depth,num_round,scale_pos_weight
0,[xgboost-poc-1],[xgboost-poc-1-0226-20431708980238],0.821841,0.821841,0.821841,0.0,0.821841,"""0.2""","""3""","""100""","""29"""
1,[xgboost-poc-1],[xgboost-poc-1-m5-lar-1-xgboost-d-0323-06241648016648],0.821841,0.821841,0.821841,0.0,0.821841,"""0.2""","""3""","""100""","""29"""
2,[xgboost-poc-1],[xgboost-poc-1-m5-lar-1-xgboost-d-0322-13081647954529],0.821841,0.821841,0.821841,0.0,0.821841,"""0.2""","""3""","""100""","""29"""
3,[xgboost-poc-1],[xgboost-poc-1-m5-lar-1-xgboost-d-0709-02451657334725],0.821841,0.821841,0.821841,0.0,0.821841,"""0.2""","""3""","""100""","""29"""
4,[xgboost-poc-1],[Default-Run-Group-xgboost-poc-1],0.821841,0.821841,0.821841,0.0,0.821841,"""0.2""","""3""","""100""","""29"""
...,...,...,...,...,...,...,...,...,...,...,...
80,[xgboost-poc-1],[xgboost-poc-1-m5-lar-1-xgboost-d-0709-02491657334980],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,[xgboost-poc-1],[Default-Run-Group-xgboost-poc-1],NaN,NaN,NaN,NaN,NaN,"""0.2""","""3""","""100""","""29"""
82,[xgboost-poc-1],[xgboost-poc-1-t3-med-1-xgboost-d-0322-11311647948686],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,[xgboost-poc-1],[Default-Run-Group-xgboost-poc-1],NaN,NaN,NaN,NaN,NaN,0.2,3.0,100.0,29.0
